In [3]:
#b2210356017 Berkay Örene
import pandas as pd
import numpy as np
from math import sqrt
from collections import Counter

data_frame = pd.read_csv("16P.csv", encoding='ISO-8859-1')
data_frame = data_frame.drop(columns=["Response Id"])
personalities = {
    "ESTJ": 0,
    "ENTJ": 1,
    "ESFJ": 2,
    "ENFJ": 3,
    "ISTJ": 4,
    "ISFJ": 5,
    "INTJ": 6,
    "INFJ": 7,
    "ESTP": 8,
    "ESFP": 9,
    "ENTP": 10,
    "ENFP": 11,
    "ISTP": 12,
    "ISFP": 13,
    "INTP": 14,
    "INFP": 15}

data_frame["Personality"] = data_frame["Personality"].map(personalities)

array = data_frame.to_numpy()
normalization_array = (array[:, :-1] - np.min(array[:, :-1]))/ (np.max(array[:, :-1]) - np.min(array[:, :-1])) # feature normalization
print(normalization_array)


In [4]:
# standart KNN algorithm
first = 0
query_people_result_list = [12000, 24000, 36000, 48000, 60000]
k_list = [1, 3, 5, 7, 9]
for query in query_people_result_list: # Cross validation. I am changing test data and train data.
    test_set = normalization_array[first:query+1, :-1]
    test_set_result = array[first:query+1, -1]
    if first == 0:    
        train_set_X = normalization_array[query+1:, :-1] 
        train_set_Y = array[query+1:, -1]
    elif first == 48000:
        train_set_X = normalization_array[:first+1, :-1]
        train_set_Y = array[:first+1, -1]
    else:
        train_set_X = np.concatenate((normalization_array[:first+1, :-1], normalization_array[query+1:, :-1])) 
        train_set_Y =  np.concatenate((array[:first+1, -1], array[query+1:, -1]))
    
    

    for i in k_list: # I am changing k in here
        k = i
        def predict(query_point): # This function is for guessing personality 
            k_nearest_labels = []
            distances = np.sqrt(np.sum((query_point - train_set_X)**2, axis=1)) 

            k_indices = np.argsort(distances)[:k]
            
            for i in k_indices:      
                k_nearest_labels.append(train_set_Y[i])   
            most_common = Counter(k_nearest_labels).most_common(1)[0][0]
            
            return most_common

        predicted_labels = [predict(query_person) for query_person in test_set]
        predictions = np.array(predicted_labels)
        print([prediction for prediction in predictions])


        # Calculating matrix confusions, accurary, precision and recall.
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        
        for personality in personalities.values():   
            for i in range(predictions.shape[0]):
                if predictions[i] == test_set_result[i]:
                    if personality == test_set_result[i]:
                        TP += 1
                    elif personality != test_set_result[i]:
                        TN += 1
                else:
                    if personality == test_set_result[i]:
                        FP += 1
                    elif personality != test_set_result[i]:
                        FN += 1
        accurary = (TP + TN)/ (TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall =  TP/(TP + FN)
        print(f"These are for k={k} and test set is between {first}-{query}")
        print("Accuray is", accurary)
        print("Precision is", precision)
        print("Recall is", recall)
        
    first = query

[11, 13, 7, 12, 3, 6, 13, 6, 1, 9, 11, 3, 15, 6, 14, 7, 9, 1, 7, 12, 1, 12, 4, 7, 11, 10, 10, 3, 13, 5, 10, 0, 10, 0, 8, 4, 12, 10, 6, 12, 6, 14, 2, 9, 8, 3, 12, 0, 13, 2, 7, 0, 0, 5, 9, 11, 2, 12, 10, 15, 11, 13, 5, 7, 10, 6, 9, 11, 5, 12, 12, 3, 8, 1, 15, 4, 6, 0, 11, 3, 12, 14, 9, 0, 3, 0, 10, 13, 9, 14, 0, 14, 10, 0, 6, 5, 7, 1, 9, 6, 1, 12, 0, 6, 8, 13, 4, 13, 4, 10, 0, 3, 2, 3, 10, 3, 2, 6, 2, 7, 10, 9, 3, 10, 12, 5, 9, 13, 0, 6, 0, 14, 2, 8, 3, 9, 5, 5, 11, 10, 7, 15, 5, 10, 9, 15, 7, 8, 8, 8, 11, 14, 15, 11, 10, 11, 12, 1, 15, 10, 5, 13, 8, 9, 13, 4, 2, 10, 10, 10, 2, 1, 3, 6, 2, 9, 10, 8, 0, 10, 13, 1, 7, 2, 6, 4, 2, 15, 8, 7, 0, 14, 6, 2, 9, 9, 3, 1, 14, 4, 2, 0, 2, 8, 5, 11, 5, 15, 7, 8, 10, 12, 8, 4, 8, 11, 4, 6, 9, 10, 10, 13, 4, 10, 11, 9, 5, 0, 7, 3, 8, 6, 1, 15, 9, 10, 7, 7, 13, 1, 9, 6, 9, 15, 4, 6, 14, 12, 7, 7, 12, 6, 6, 12, 3, 5, 5, 5, 0, 0, 1, 6, 11, 14, 11, 6, 2, 6, 11, 4, 12, 4, 2, 1, 14, 8, 6, 10, 4, 15, 11, 15, 1, 11, 1, 10, 4, 7, 8, 0, 13, 3, 6, 0, 0, 14, 6, 5

In [5]:
# this part weighted KNN algorithm
# I couldn't finished printing the result for that bonus point part(...with weighted KNN algorithm) but i printed some of them and it is done for coding. 
first = 0
query_people_result_list = [12000, 24000, 36000, 48000, 60000]
k_list = [1, 3, 5, 7, 9]
for query in query_people_result_list: # Cross validation. I am changing test data and train data.
    test_set = normalization_array[first:query+1, :-1]
    test_set_result = array[first:query+1, -1]
    if first == 0:    
        train_set_X = normalization_array[query+1:, :-1] 
        train_set_Y = array[query+1:, -1]
    elif first == 48000:
        train_set_X = normalization_array[:first+1, :-1]
        train_set_Y = array[:first+1, -1]
    else:
        train_set_X = np.concatenate((normalization_array[:first+1, :-1], normalization_array[query+1:, :-1])) 
        train_set_Y =  np.concatenate((array[:first+1, -1], array[query+1:, -1]))
    
    

    for i in k_list: # I am changing k in here
        k = i
        def predict(query_point):
            k_nearest_distance_and_labels  = []
            distances = np.sqrt(np.sum((query_point - train_set_X)**2, axis=1))  

            k_indices = np.argsort(distances)[:k]
            
            for i in k_indices:      
                k_nearest_distance_and_labels.append((distances[i], train_set_Y[i]))   
            
            labeled_weight_sums = {}
            for neighbor in k_nearest_distance_and_labels:
                if neighbor[1] in labeled_weight_sums:
                    labeled_weight_sums[neighbor[1]] += 1/neighbor[0]**2
                else:
                    labeled_weight_sums[neighbor[1]] = 1/neighbor[0]**2
        
            label = None
            max_weight_sum = max(labeled_weight_sums.values())
            for key in labeled_weight_sums.keys():
                if labeled_weight_sums[key] == max_weight_sum:
                    label = key
                    break
        
            return key
            
        
        #query_people_result = array[59990:, -1]
        predicted_labels = [predict(query_person) for query_person in test_set]
        predictions = np.array(predicted_labels)
        
        print([prediction for prediction in predictions])

        
        # Calculating matrix confusions, accurary, precision and recall.
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        
        for personality in personalities.values():   
            for i in range(predictions.shape[0]):
                if predictions[i] == test_set_result[i]:
                    if personality == test_set_result[i]:
                        TP += 1
                    elif personality != test_set_result[i]:
                        TN += 1
                else:
                    if personality == test_set_result[i]:
                        FP += 1
                    elif personality != test_set_result[i]:
                        FN += 1
        accurary = (TP + TN)/ (TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall =  TP/(TP + FN)
        print(f"These are for k={k} and test set is between {first}-{query}")
        print("Accuray is", accurary)
        print("Precision is", precision)
        print("Recall is", recall)
        
    first = query

[11, 13, 7, 12, 3, 6, 13, 6, 1, 9, 11, 3, 15, 6, 14, 7, 9, 1, 7, 12, 1, 12, 4, 7, 11, 10, 10, 3, 13, 5, 10, 0, 10, 0, 8, 4, 12, 10, 6, 12, 6, 14, 2, 9, 8, 3, 12, 0, 13, 2, 7, 0, 0, 5, 9, 11, 2, 12, 10, 15, 11, 13, 5, 7, 10, 6, 9, 11, 5, 12, 12, 3, 8, 1, 15, 4, 6, 0, 11, 3, 12, 14, 9, 0, 3, 0, 10, 13, 9, 14, 0, 14, 10, 0, 6, 5, 7, 1, 9, 6, 1, 12, 0, 6, 8, 13, 4, 13, 4, 10, 0, 3, 2, 3, 10, 3, 2, 6, 2, 7, 10, 9, 3, 10, 12, 5, 9, 13, 0, 6, 0, 14, 2, 8, 3, 9, 5, 5, 11, 10, 7, 15, 5, 10, 9, 15, 7, 8, 8, 8, 11, 14, 15, 11, 10, 11, 12, 1, 15, 10, 5, 13, 8, 9, 13, 4, 2, 10, 10, 10, 2, 1, 3, 6, 2, 9, 10, 8, 0, 10, 13, 1, 7, 2, 6, 4, 2, 15, 8, 7, 0, 14, 6, 2, 9, 9, 3, 1, 14, 4, 2, 0, 2, 8, 5, 11, 5, 15, 7, 8, 10, 12, 8, 4, 8, 11, 4, 6, 9, 10, 10, 13, 4, 10, 11, 9, 5, 0, 7, 3, 8, 6, 1, 15, 9, 10, 7, 7, 13, 1, 9, 6, 9, 15, 4, 6, 14, 12, 7, 7, 12, 6, 6, 12, 3, 5, 5, 5, 0, 0, 1, 6, 11, 14, 11, 6, 2, 6, 11, 4, 12, 4, 2, 1, 14, 8, 6, 10, 4, 15, 11, 15, 1, 11, 1, 10, 4, 7, 8, 0, 13, 3, 6, 0, 0, 14, 6, 5

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22988\854695002.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  labeled_weight_sums[neighbor[1]] = 1/neighbor[0]**2


KeyboardInterrupt: 